In [1]:
# Parameters
celltype = "Plasma"


In [2]:
for v in ['celltype']:
    if (v not in locals()) or (v not in globals()):
        raise TypeError(f"{v} not defined")
    else:
        print(f"{v} = {eval(v)}")

celltype = Plasma


In [3]:
# General modules
import sys
import os
import session_info
import warnings
from pyprojroot.here import here
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as plines
import matplotlib.colors as mcolors
from IPython.display import display
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.lines import Line2D  # for legend handle

# Specific modules
import scanpy as sc
import decoupler as dc
import anndata as ad
import scienceplots

# Setting some parameters
warnings.filterwarnings("ignore")

# Import functions
sys.path.insert(1, str(here('bin')))
from customPalette import *
from customPythonFunctions import mean_by_category, filter_low_represented_cell_group, RelativeDiff_mean_by_category, mscatter, aggregating_features

plt.style.use(['nature'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

overwriteFigures = True
overwriteData = True

pd.options.display.max_columns = 300
pd.options.display.max_rows = 1000

In [4]:
print("Main directory path: {}".format(here()))

Main directory path: /scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas


**Load data**

In [5]:
inputfile_path = here("03_downstream_analysis/04_integration_with_annotation/results/normalized_adatas_nextflow/cellType_adata_merged/{}_adataMerged.log1p.h5ad".format(celltype))
# Loading scanpy (Anndata) object
adata = sc.read_h5ad(inputfile_path)

# Revert log1p
adata.X = np.expm1(adata.X)
adata

AnnData object with n_obs × n_vars = 16854 × 8253
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'Level1', 'Level2'
    uns: 'log1p'

**Generate new variables**

In [6]:
# Generate new variables for CELL level1
adata.obs["disease_level1"] = ["{}__{}".format(disease, level1)
                                           for disease, level1 in zip(adata.obs["disease"], adata.obs["Level1"])]
adata.obs["disease_level1"] = adata.obs["disease_level1"].astype('category')

adata.obs["disease_studyID"] = ["{}_{}".format(disease, studyID)
                                           for disease, studyID in zip(adata.obs["disease"], adata.obs["studyID"])]
adata.obs["disease_studyID_level1"] = ["{}__{}".format(adata.obs["disease_studyID"][i], level1) if disease == "healthy"
                                                  else "{}__{}".format(disease, level1) 
                                                  for i, (disease, level1) 
                                                  in enumerate(zip(adata.obs["disease"], adata.obs["Level1"]))]
adata.obs["disease_studyID_level1"] = adata.obs["disease_studyID_level1"].astype('category')


# Generate new variables for CELL TYPE  
adata.obs["disease_level2"] = ["{}__{}".format(disease, level2)
                                           for disease, level2 in zip(adata.obs["disease"], adata.obs["Level2"])]
adata.obs["disease_level2"] = adata.obs["disease_level2"].astype('category')

adata.obs["disease_studyID"] = ["{}_{}".format(disease, studyID)
                                           for disease, studyID in zip(adata.obs["disease"], adata.obs["studyID"])]
adata.obs["disease_studyID_level2"] = ["{}__{}".format(adata.obs["disease_studyID"][i], level2) if disease == "healthy"
                                                  else "{}__{}".format(disease, level2) 
                                                  for i, (disease, level2) 
                                                  in enumerate(zip(adata.obs["disease"], adata.obs["Level2"]))]
adata.obs["disease_studyID_level2"] = adata.obs["disease_studyID_level2"].astype('category')

adata

AnnData object with n_obs × n_vars = 16854 × 8253
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'Level1', 'Level2', 'disease_level1', 'disease_studyID', 'disease_studyID_level1', 'disease_level2', 'disease_studyID_level2'
    uns: 'log1p'

**Load SPECTRA Gene Weights**

In [7]:
geneWeights = pd.read_csv(here('03_downstream_analysis/05_SPECTRA/results/SPECTRAFactor_Renamed_Processed_LAM0.001_S0.6_GeneWeights.csv'), 
                          index_col='Unnamed: 0')
geneWeights.head()

,ENSG00000000003,ENSG00000000457,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461,ENSG00000001561,ENSG00000001630,ENSG00000001631,ENSG00000002016,ENSG00000002330,ENSG00000002549,ENSG00000002586,ENSG00000002587,ENSG00000002822,ENSG00000002933,ENSG00000003147,ENSG00000003249,ENSG00000003400,ENSG00000003402,ENSG00000003436,ENSG00000004059,ENSG00000004139,ENSG00000004142,ENSG00000004399,ENSG00000004468,ENSG00000004478,ENSG00000004487,ENSG00000004700,ENSG00000004766,ENSG00000004799,ENSG00000004846,ENSG00000004939,ENSG00000004975,ENSG00000005007,ENSG00000005020,ENSG00000005059,ENSG00000005102,ENSG00000005187,ENSG00000005206,ENSG00000005243,ENSG00000005249,ENSG00000005339,ENSG00000005379,ENSG00000005381,ENSG00000005436,ENSG00000005469,ENSG00000005471,ENSG00000005486,ENSG00000005812,ENSG00000005844,ENSG00000005882,ENSG00000005884,ENSG00000005961,ENSG00000006015,ENSG00000006025,ENSG00000006125,ENSG00000006194,ENSG00000006210,ENSG00000006282,ENSG00000006327,ENSG00000006432,ENSG00000006459,ENSG00000006530,ENSG00000006534,ENSG00000006555,ENSG00000006576,ENSG00000006607,ENSG00000006625,ENSG00000006634,ENSG00000006638,ENSG00000006652,ENSG00000006715,ENSG00000006740,ENSG00000006757,ENSG00000006837,ENSG00000007038,ENSG00000007047,ENSG00000007129,ENSG00000007202,ENSG00000007237,ENSG00000007255,ENSG00000007264,ENSG00000007312,ENSG00000007350,ENSG00000007376,ENSG00000007392,ENSG00000007402,ENSG00000007516,ENSG00000007520,ENSG00000007541,ENSG00000007545,ENSG00000007944,ENSG00000007968,ENSG00000008056,ENSG00000008083,ENSG00000008086,ENSG00000008130,ENSG00000008226,ENSG00000008256,ENSG00000008277,ENSG00000008283,ENSG00000008294,ENSG00000008300,ENSG00000008311,ENSG00000008382,ENSG00000008394,ENSG00000008441,ENSG00000008513,ENSG00000008517,ENSG00000008710,ENSG00000008869,ENSG00000009335,ENSG00000009413,ENSG00000009694,ENSG00000009790,ENSG00000009950,ENSG00000010030,ENSG00000010165,ENSG00000010219,ENSG00000010270,ENSG00000010278,ENSG00000010292,ENSG00000010295,ENSG00000010310,ENSG00000010318,ENSG00000010319,ENSG00000010327,ENSG00000010404,ENSG00000010539,ENSG00000010610,ENSG00000010671,ENSG00000010704,ENSG00000010803,ENSG00000010810,ENSG00000010818,ENSG00000011028,ENSG00000011105,ENSG00000011114,ENSG00000011143,ENSG00000011198,ENSG00000011258,ENSG00000011295,ENSG00000011376,ENSG00000011422,ENSG00000011451,ENSG00000011523,...,ENSG00000258366,ENSG00000258429,ENSG00000258472,ENSG00000258484,ENSG00000258659,ENSG00000258818,ENSG00000258839,ENSG00000258890,ENSG00000259207,ENSG00000259431,ENSG00000259956,ENSG00000260001,ENSG00000260314,ENSG00000260456,ENSG00000261052,ENSG00000261150,ENSG00000261221,ENSG00000261236,ENSG00000261371,ENSG00000261609,ENSG00000262246,ENSG00000262655,ENSG00000262814,ENSG00000262919,ENSG00000263002,ENSG00000263155,ENSG00000263465,ENSG00000263528,ENSG00000263874,ENSG00000263961,ENSG00000264343,ENSG00000264364,ENSG00000264522,ENSG00000265491,ENSG00000265808,ENSG00000265972,ENSG00000266028,ENSG00000266094,ENSG00000266173,ENSG00000266338,ENSG00000266412,ENSG00000266714,ENSG00000266964,ENSG00000266967,ENSG00000267041,ENSG00000267368,ENSG00000267534,ENSG00000267673,ENSG00000267680,ENSG00000267796,ENSG00000267855,ENSG00000268350,ENSG00000269028,ENSG00000269313,ENSG00000269343,ENSG00000269404,ENSG00000269713,ENSG00000269858,ENSG00000270170,ENSG00000270629,ENSG00000270672,ENSG00000271254,ENSG00000271303,ENSG00000271383,ENSG00000271425,ENSG00000271447,ENSG00000271503,ENSG00000271601,ENSG00000271605,ENSG00000272047,ENSG00000272325,ENSG00000272333,ENSG00000272391,ENSG00000272398,ENSG00000272602,ENSG00000272886,ENSG00000273045,ENSG00000273136,ENSG00000273173,ENSG00000273274,ENSG00000273748,ENSG00000273802,ENSG00000273899,ENSG00000274070,ENSG00000274180,ENSG00000274211,ENSG00000274290,ENSG00000274349,ENSG00000274523,ENSG00000274588,ENSG00000274618,ENSG00000274736,ENSG00000274847,ENSG00000274997,ENSG00000275004,ENSG00000275023,ENSG00000275063,ENSG00000275066,ENSG00000275074,ENSG0000027

In [8]:
# Subset to consider only GLOBAL & cell-type specific

geneWeights['SPECTRA_CellTypeFactor'] = geneWeights.index.str.split('-X-').str[1]
geneWeights = geneWeights[geneWeights['SPECTRA_CellTypeFactor'].isin(['global'] + [celltype])]
geneWeights.drop(columns=['SPECTRA_CellTypeFactor'], inplace=True)
geneWeights

,ENSG00000000003,ENSG00000000457,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461,ENSG00000001561,ENSG00000001630,ENSG00000001631,ENSG00000002016,ENSG00000002330,ENSG00000002549,ENSG00000002586,ENSG00000002587,ENSG00000002822,ENSG00000002933,ENSG00000003147,ENSG00000003249,ENSG00000003400,ENSG00000003402,ENSG00000003436,ENSG00000004059,ENSG00000004139,ENSG00000004142,ENSG00000004399,ENSG00000004468,ENSG00000004478,ENSG00000004487,ENSG00000004700,ENSG00000004766,ENSG00000004799,ENSG00000004846,ENSG00000004939,ENSG00000004975,ENSG00000005007,ENSG00000005020,ENSG00000005059,ENSG00000005102,ENSG00000005187,ENSG00000005206,ENSG00000005243,ENSG00000005249,ENSG00000005339,ENSG00000005379,ENSG00000005381,ENSG00000005436,ENSG00000005469,ENSG00000005471,ENSG00000005486,ENSG00000005812,ENSG00000005844,ENSG00000005882,ENSG00000005884,ENSG00000005961,ENSG00000006015,ENSG00000006025,ENSG00000006125,ENSG00000006194,ENSG00000006210,ENSG00000006282,ENSG00000006327,ENSG00000006432,ENSG00000006459,ENSG00000006530,ENSG00000006534,ENSG00000006555,ENSG00000006576,ENSG00000006607,ENSG00000006625,ENSG00000006634,ENSG00000006638,ENSG00000006652,ENSG00000006715,ENSG00000006740,ENSG00000006757,ENSG00000006837,ENSG00000007038,ENSG00000007047,ENSG00000007129,ENSG00000007202,ENSG00000007237,ENSG00000007255,ENSG00000007264,ENSG00000007312,ENSG00000007350,ENSG00000007376,ENSG00000007392,ENSG00000007402,ENSG00000007516,ENSG00000007520,ENSG00000007541,ENSG00000007545,ENSG00000007944,ENSG00000007968,ENSG00000008056,ENSG00000008083,ENSG00000008086,ENSG00000008130,ENSG00000008226,ENSG00000008256,ENSG00000008277,ENSG00000008283,ENSG00000008294,ENSG00000008300,ENSG00000008311,ENSG00000008382,ENSG00000008394,ENSG00000008441,ENSG00000008513,ENSG00000008517,ENSG00000008710,ENSG00000008869,ENSG00000009335,ENSG00000009413,ENSG00000009694,ENSG00000009790,ENSG00000009950,ENSG00000010030,ENSG00000010165,ENSG00000010219,ENSG00000010270,ENSG00000010278,ENSG00000010292,ENSG00000010295,ENSG00000010310,ENSG00000010318,ENSG00000010319,ENSG00000010327,ENSG00000010404,ENSG00000010539,ENSG00000010610,ENSG00000010671,ENSG00000010704,ENSG00000010803,ENSG00000010810,ENSG00000010818,ENSG00000011028,ENSG00000011105,ENSG00000011114,ENSG00000011143,ENSG00000011198,ENSG00000011258,ENSG00000011295,ENSG00000011376,ENSG00000011422,ENSG00000011451,ENSG00000011523,...,ENSG00000258366,ENSG00000258429,ENSG00000258472,ENSG00000258484,ENSG00000258659,ENSG00000258818,ENSG00000258839,ENSG00000258890,ENSG00000259207,ENSG00000259431,ENSG00000259956,ENSG00000260001,ENSG00000260314,ENSG00000260456,ENSG00000261052,ENSG00000261150,ENSG00000261221,ENSG00000261236,ENSG00000261371,ENSG00000261609,ENSG00000262246,ENSG00000262655,ENSG00000262814,ENSG00000262919,ENSG00000263002,ENSG00000263155,ENSG00000263465,ENSG00000263528,ENSG00000263874,ENSG00000263961,ENSG00000264343,ENSG00000264364,ENSG00000264522,ENSG00000265491,ENSG00000265808,ENSG00000265972,ENSG00000266028,ENSG00000266094,ENSG00000266173,ENSG00000266338,ENSG00000266412,ENSG00000266714,ENSG00000266964,ENSG00000266967,ENSG00000267041,ENSG00000267368,ENSG00000267534,ENSG00000267673,ENSG00000267680,ENSG00000267796,ENSG00000267855,ENSG00000268350,ENSG00000269028,ENSG00000269313,ENSG00000269343,ENSG00000269404,ENSG00000269713,ENSG00000269858,ENSG00000270170,ENSG00000270629,ENSG00000270672,ENSG00000271254,ENSG00000271303,ENSG00000271383,ENSG00000271425,ENSG00000271447,ENSG00000271503,ENSG00000271601,ENSG00000271605,ENSG00000272047,ENSG00000272325,ENSG00000272333,ENSG00000272391,ENSG00000272398,ENSG00000272602,ENSG00000272886,ENSG00000273045,ENSG00000273136,ENSG00000273173,ENSG00000273274,ENSG00000273748,ENSG00000273802,ENSG00000273899,ENSG00000274070,ENSG00000274180,ENSG00000274211,ENSG00000274290,ENSG00000274349,ENSG00000274523,ENSG00000274588,ENSG00000274618,ENSG00000274736,ENSG00000274847,ENSG00000274997,ENSG00000275004,ENSG00000275023,ENSG00000275063,ENSG00000275066,ENSG00000275074,ENSG0000027

**Load SPECTRA GeneSet Factors**

**Converting 'geneWeights' to match with DecoupleR 'net' format**

In [9]:
geneWeights_netDF = geneWeights.reset_index().melt(id_vars='index', var_name='source', value_name='weight')
geneWeights_netDF = geneWeights_netDF.rename(columns={'index': 'source', 'source':'target'})
geneWeights_netDF.head()

,source,target,weight
0,0-X-global-X-all_IL6-JAK-STAT3_signaling,ENSG00000000003,1.662220e-15
1,1-X-global-X-all_JAK-STAT_signaling,ENSG00000000003,1.108171e-15
2,2-X-global-X-all_type-IandII-ifn-response,ENSG00000000003,2.737120e-15
3,3-X-global-X-all_TNF-via-NFkB_signaling,ENSG00000000003,8.518205e-16
4,4-X-global-X-all_MHC-II-Ipresentation,ENSG00000000003,2.003873e-15


In [10]:
# Count the number of GENES of each FACTOR
source_counts = geneWeights_netDF['source'].value_counts()

# Get the sources with counts less than 5
sources_to_drop = source_counts[source_counts < 5].index
print(sources_to_drop)

# Filter geneWeights DataFrame if source has counts less than 5
geneWeights_netDF = geneWeights_netDF[~geneWeights_netDF['source'].isin(sources_to_drop)]

Index([], dtype='object', name='source')


# Run DecoupleR - Computing gene signature with ULM

https://decoupler-py.readthedocs.io/en/latest/notebooks/progeny.html#Activity-inference-with-multivariate-linear-model-(ulm)

To infer pathway enrichment scores we will run the univariate linear model (ulm)  method. For each cell in our dataset (adata), it fits a linear model that predicts the observed gene expression based on all pathways’ Pathway-Gene interactions weights. Once fitted, the obtained t-values of the slopes are the scores. If it is positive, we interpret that the pathway is active and if it is negative we interpret that it is inactive. Since genes might be overlapping across gene sets (signatures) it is better to use ULM methods.

To run `decoupler` methods, we need an input matrix (`mat`), an input prior knowledge network/restudyID (`net`), and the name of the columns of net that we want to use.

* The obtained scores (t-values)(`ulm_estimate`) and p-values (`ulm_pvals`) are stored in the `adata.obsm` key.
* Note: Each run of run_ulm overwrites what is inside of `ulm_estimate` and `ulm_pvals`.

In [11]:
# Generating pseudobulks

adataPS = aggregating_features(adata.X, adata.obs[['studyID','sampleID', 'chemistry', 'disease', 'sex','binned_age', 'Level1']], #, 'sex', 'binned_age']]
                              mode = 'mean', obs_names_col = ['sampleID', 'Level1'], min_observation = 10)
adataPS.var_names = adata.var_names
adataPS

AnnData object with n_obs × n_vars = 347 × 8253
    obs: 'studyID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'Level1', 'n_observation'

In [12]:
sc.pp.log1p(adataPS)
adataPS.layers["scANVI_log1p_10e4_counts_scaled"] = adataPS.X.copy()

#zscaling
sc.pp.scale(adataPS, layer="scANVI_log1p_10e4_counts_scaled")
adataPS.X = adataPS.layers["scANVI_log1p_10e4_counts_scaled"]

adataPS

AnnData object with n_obs × n_vars = 347 × 8253
    obs: 'studyID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'Level1', 'n_observation'
    var: 'mean', 'std'
    uns: 'log1p'
    layers: 'scANVI_log1p_10e4_counts_scaled'

## Gene activity inference with Univariate Linear Model (ULM)

In [13]:
dc.run_ulm(
    mat=adataPS,
    net=geneWeights_netDF,
    source='source',
    target='target',
    weight='weight',
    verbose=True,
    use_raw=False, # Use raw attribute of mat if present.
    min_n = 5 # Minimum of targets per studyID. If less, studyIDs are removed. Default 5, but having a studyID of 3.
)

Running ulm on mat with 347 samples and 8253 targets for 15 sources.


In [14]:
adataPS

AnnData object with n_obs × n_vars = 347 × 8253
    obs: 'studyID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'Level1', 'n_observation'
    var: 'mean', 'std'
    uns: 'log1p'
    obsm: 'ulm_estimate', 'ulm_pvals'
    layers: 'scANVI_log1p_10e4_counts_scaled'

In [15]:
# Save results
file_path = here('03_downstream_analysis/06_inflammation_signatures/results/DecoupleR_ulmestimates_{}_L1_Corr.csv'.format(celltype))
adataPS.obsm['ulm_estimate'].to_csv(file_path, index=True) 

file_path = here('03_downstream_analysis/06_inflammation_signatures/results/DecoupleR_ulmpvals_{}_L1_Corr.csv'.format(celltype))
adataPS.obsm['ulm_pvals'].to_csv(file_path, index=True) 

# Explore ulm results

To visualize the obtained scores, we can re-use many of `scanpy`’s plotting functions. First though, we need to extract the activities from the adata object.

`dc.get_acts` returns a new AnnData object which holds the obtained activities in its `adataPS.X` attribute, allowing us to re-use many scanpy functions, for example let’s visualise the Trail pathway:

In [16]:
adataPS = dc.get_acts(adataPS, obsm_key='ulm_estimate')
adataPS

AnnData object with n_obs × n_vars = 347 × 15
    obs: 'studyID', 'sampleID', 'chemistry', 'disease', 'sex', 'binned_age', 'Level1', 'n_observation'
    uns: 'log1p'
    obsm: 'ulm_estimate', 'ulm_pvals'

In [17]:
# Save results
anndata_path = here('03_downstream_analysis/06_inflammation_signatures/results/DecoupleR_ulmestimates_{}_L1_Corr.h5ad'.format(celltype))
adataPS.write_h5ad(anndata_path)

In [18]:
session_info.show()